In [ ]:
import os, pydeck as pdk, pandas as pd, numpy as np
from datetime import date, datetime, timedelta
from sqlalchemy import create_engine

- pings city
- contacts city
- contacts selected events
- selection bias: home locations

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## database connection

In [ ]:
#Connecting to the aws
# database credentials
db_usr, db_pwd = os.getenv('DB_USR'), os.getenv('DB_PWD') # your database user name and password
# database login
host, port, db = 'nc-health-data-prod.cluster-ccsgl7rk4urn.eu-central-1.rds.amazonaws.com', 5432, 'master'
engine = create_engine('postgresql://'+db_usr+':'+db_pwd+'@'+host+':'+str(port)+'/'+db)
conn = engine.connect()

In [ ]:
conn.close()

## pings

In [ ]:
city = 'Stuttgart'
day = '2024-06-28'#'2024-07-05'#'2024-06-28'

In [ ]:
city = 'Düsseldorf'
day = '2024-06-01'#'2024-06-01'#'2024-05-25'

In [ ]:
query = f"""
    with restricted as (
    	select ex.*
    	from tuberlin_euro24_tileid as ti
    	join ex_corona_sdkv6 as ex on ex.tile_id = ti.tile_id
    	where "day" = '{day}' -- between '{di}' and '{df}'
    ),
    ex_tmptmp as (
        select
                  "day"
                , stime
                , did
                , tl8
                , tile_id
            from restricted, unnest(stime_arr, tile_arr) u(stime, tl8)
    ),
    ex_tmp as (
        select
                  "day"
                , stime
                , did
                , st_transform(st_translate(st_setsrid(tile8togeo(tl8), 32632), tx.minx, tx.miny), 3857) as geopoint
                , tl8
            from ex_tmptmp
            join txc_dt_grid_1000m as tx on ex_tmptmp.tile_id = tx.tile_id
    ),
    cities2 as (
        select osm_id, "name", max(way_area) as way_area
        from tuberlin_euro24_contour
        where "name" = '{city}'
        group by 1,2
    ),
    cities3 as (
        select c1."name", c1.way
        from tuberlin_euro24_contour as c1
        join cities2 as c2 on c1.way_area = c2.way_area
    ),
    ex as (
        select
                  "day"
                , stime
                , did
                , "name" as city
                --, geopoint
                , st_x(st_transform(geopoint, 4326)) as lon, st_y(st_transform(geopoint, 4326)) as lat
                , tl8
        from ex_tmp
        join cities3 as c3 on st_contains(c3.way, ex_tmp.geopoint)
    )
    select *
    from ex
"""

In [ ]:
data = pd.DataFrame(pd.read_sql_query(query, conn))
data

In [ ]:
data['size'] = 10

In [ ]:
viewstate = {'Stuttgart': [48.78422258241185, 9.18497909163697], 'Düsseldorf': [51.224949154286364, 6.780327227212697]}

In [ ]:
scatter = pdk.Layer(
    "ScatterplotLayer",
    data[['lon','lat','size']],
    get_position=["lon", "lat"],  # Must match column names
    get_radius="size",
    get_fill_color=[255, 0, 0, 160],
    pickable=True
)

view_state = pdk.ViewState(
    #longitude=data.lon.mean(), latitude=data.lat.mean(),
    longitude=viewstate[city][1], latitude=viewstate[city][0],
    zoom=12, pitch=0
)

r = pdk.Deck(layers=[scatter], initial_view_state=view_state, map_style="https://tiles.stadiamaps.com/styles/osm_bright.json")#.show()
# carto light: "https://basemaps.cartocdn.com/gl/positron-gl-style/style.json"
# carto dark: "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json"
# carto voyager: "https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json"
# osm: "https://tiles.stadiamaps.com/styles/osm_bright.json"
r.to_html("plots/hexagon_layer.html")

In [ ]:
from selenium import webdriver
#from PIL import Image
import time

# Path to ChromeDriver (or use another browser driver)
driver = webdriver.Chrome()

# Open the HTML file
driver.get("C:/Users/steven.schulz/Documents/nc-euro24/hexagon_layer.html")  # Replace with the actual path
time.sleep(3)  # Wait for rendering

# Capture screenshot
screenshot_path = "hexagon_layer.png"
driver.save_screenshot(screenshot_path)

# Close the browser
driver.quit()

# Optionally, convert to JPG
#img = Image.open(screenshot_path)
#img.convert("RGB").save("map.jpg", "JPEG")

print("Map saved as map.jpg")

In [ ]:
tl82w = data.groupby('tl8').did.apply(len).reset_index().rename(columns={'did':'weight'})
data = data.merge(tl82w, on='tl8')
data['weight_log'] = 1.+data.weight.apply(np.log)
data

In [ ]:
COLOR_BREWER_RED_SCALE = [
    [255, 245, 240],  # Lightest Red (similar to light blue)
    [252, 224, 210],
    [250, 187, 161],
    [245, 144, 120],
    [240, 82, 79],
    [165, 0, 38],  # Deepest Red (similar to deep blue)
]


scatter = pdk.Layer(
    "HeatmapLayer",
    data[['lon','lat','weight_log']],
    get_position=["lon", "lat"],  # Must match column names
    opacity=0.9,
    aggregation=pdk.types.String("MEAN"),
    color_range=COLOR_BREWER_RED_SCALE,
    threshold=.05,
    radiusPixels=10,
    intensity=1.5,
    get_weight="weight_log",
    pickable=True,
)

view_state = pdk.ViewState(
    longitude=data.lon.mean(), latitude=data.lat.mean(), zoom=12, pitch=0
)

#r = pdk.Deck(layers=[scatter], initial_view_state=view_state)#.show()
r = pdk.Deck(
    layers=[scatter],
    initial_view_state=view_state,
    map_style="https://tiles.stadiamaps.com/styles/osm_bright.json",
    #map_provider="mapbox",
    #map_style=pdk.map_styles.SATELLITE,
    #tooltip={"text": "Concentration of cattle in blue, concentration of poultry in orange"},
)#.show()
r.to_html("plots/hexagon_layer.html")

In [ ]:
# Define a layer to display on a map
layer = pdk.Layer(
    "HexagonLayer",
    data[['lon','lat']],
    get_position=["lon", "lat"],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 300],
    extruded=True,
    coverage=1,
    opacity=.5,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=data.lon.mean(),
    latitude=data.lat.mean(),
    zoom=9,
    min_zoom=9,
    max_zoom=15,
    pitch=0,#40.5,
    bearing=0,#-27.36,
)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, map_style="https://tiles.stadiamaps.com/styles/osm_bright.json",)
r.to_html("plots/hexagon_layer.html")

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/geojson/vancouver-blocks.json"
LAND_COVER = [[[-123.0, 49.196], [-123.0, 49.324], [-123.306, 49.324], [-123.306, 49.196]]]

INITIAL_VIEW_STATE = pydeck.ViewState(
  latitude=49.254,
  longitude=-123.13,
  zoom=11,
  max_zoom=16,
  pitch=45,
  bearing=0
)

polygon = pydeck.Layer(
    'PolygonLayer',
    LAND_COVER,
    stroked=False,
    # processes the data as a flat longitude-latitude pair
    get_polygon='-',
    get_fill_color=[0, 0, 0, 20]
)

geojson = pydeck.Layer(
    'GeoJsonLayer',
    DATA_URL,
    opacity=0.8,
    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    get_elevation='properties.valuePerSqm / 20',
    get_fill_color='[255, 255, properties.growth * 255]',
    get_line_color=[255, 255, 255],
    pickable=True
)

r = pydeck.Deck(
    layers=[polygon, geojson],
    initial_view_state=INITIAL_VIEW_STATE)

r.to_html()